In [ ]:
import pandas as pd

In [ ]:
df_business = pd.read_json('yelp_academic_dataset_business.json',lines=True)

# Finding all categories in LA

In [ ]:
df=df_business.loc[df_business['city'].str.contains('Las Vegas')]

# Extract categories for filtering

In [ ]:
def findCat(df):
    if df.categories != None:
        listOfCat=df.categories.split(',')
        return listOfCat
    else:
        return []

In [ ]:
categories=[]
for i in range(len(df)):
    categories = categories + findCat(df.iloc[i])

In [ ]:
categories.sort()

In [ ]:
res = []
for each in categories:
    if each not in res:
        res.append(each)

In [ ]:
df2 = pd.DataFrame()

In [ ]:
df2['categories']=res

In [ ]:
df2.to_csv('categories.csv')

# from here manually filter categories belonging to food.
# list of food categories in 'categories_filtered.csv'

# Finding business that are food

In [ ]:
# Extract all categories_filtered into a list

df_categories = pd.read_csv('categories_filtered.csv',index_col=0)

catlist = []
for i in range(len(df_categories)+1):
    if i!=29:
        catlist.append(df_categories['categories'][i])

# extracting rows that contains categories filtered

In [ ]:
def findCat(x):
    res = []
    if x != None:
        xsplit = x.split(',')
        for each in xsplit:
            if each.strip() in catlist:
                res.append(each)
        if len(res)==0:
            return 0
        else: return res
    else:
        return 0

In [ ]:
df2=pd.DataFrame(df['categories'].apply(lambda x:findCat(x)))

In [ ]:
indexes=df2[df2['categories']!=0].index

In [ ]:
columnsWeNeed = ['business_id', 'latitude', 'longitude', 'stars', 'review_count', 'categories']

df_out = df_business.iloc[indexes].loc[:,columnsWeNeed].reset_index(drop=True)

In [ ]:
df_out

,business_id,latitude,longitude,stars,review_count,categories
0,fnZrZlqW1Z8iWgTVDfv_MA,36.099738,-115.301568,2.5,15,"Mexican, Restaurants, Fast Food"
1,fhNf_sg-XzZ3e7HEVGuOZg,36.071196,-115.207463,3.0,10,"Burgers, Restaurants, American (Traditional), ..."
2,OLciBerMDmD_WYzfYbmkqA,36.110799,-115.172507,3.5,4,"Nightlife, Arts & Entertainment, Bars, Lounges..."
3,OWkS1FXNJbozn-qPg3LWxg,36.128561,-115.171130,4.5,11,"Food, Food Trucks, Restaurants, Pizza"
4,MTx-Zdl_KcU_z9G832XAjg,36.307495,-115.279059,2.5,37,"Fast Food, Restaurants"
...,...,...,...,...,...,...
10897,tv-WIUQifcp5J9pUKoiE3Q,36.076518,-115.153343,4.0,69,"Food, Asian Fusion, Food Trucks, Restaurants"
10898,kGVAle6VpDvZHlEeTB2WhA,36.035739,-115.171471,4.0,10,"Delis, Restaurants, Sandwiches, Food, Pizza"
10899,aJK0MWXKgXIgQUCM9fqSJQ,36.129782,-115.274674,3.5,69,"Pizza, Italian, Restaurants"
10900,5VXlHFrYZZzj7zfZEsz5nQ,36.105289,-115.174037,3.0,279,"Chinese, Restaurants"


In [ ]:
df_out.to_csv('yelp_business.csv')

# exploration

In [ ]:
df = pd.read_csv('yelp_business.csv')

In [ ]:
df

,Unnamed: 0,business_id,latitude,longitude,stars,review_count,categories
0,0,fnZrZlqW1Z8iWgTVDfv_MA,36.099738,-115.301568,2.5,15,"Mexican, Restaurants, Fast Food"
1,1,fhNf_sg-XzZ3e7HEVGuOZg,36.071196,-115.207463,3.0,10,"Burgers, Restaurants, American (Traditional), ..."
2,2,OLciBerMDmD_WYzfYbmkqA,36.110799,-115.172507,3.5,4,"Nightlife, Arts & Entertainment, Bars, Lounges..."
3,3,OWkS1FXNJbozn-qPg3LWxg,36.128561,-115.171130,4.5,11,"Food, Food Trucks, Restaurants, Pizza"
4,4,MTx-Zdl_KcU_z9G832XAjg,36.307495,-115.279059,2.5,37,"Fast Food, Restaurants"
...,...,...,...,...,...,...,...
10897,10897,tv-WIUQifcp5J9pUKoiE3Q,36.076518,-115.153343,4.0,69,"Food, Asian Fusion, Food Trucks, Restaurants"
10898,10898,kGVAle6VpDvZHlEeTB2WhA,36.035739,-115.171471,4.0,10,"Delis, Restaurants, Sandwiches, Food, Pizza"
10899,10899,aJK0MWXKgXIgQUCM9fqSJQ,36.129782,-115.274674,3.5,69,"Pizza, Italian, Restaurants"
10900,10900,5VXlHFrYZZzj7zfZEsz5nQ,36.105289,-115.174037,3.0,279,"Chinese, Restaurants"
